In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
import logging
# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [27]:
# スクレイピング対象のURL
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC005/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13104&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0'

# requestsでURLからデータを取得
response = requests.get(url)
response.encoding = response.apparent_encoding

# BeautifulSoupオブジェクトの生成
soup = BeautifulSoup(response.text, 'html.parser')


In [28]:
# 物件情報を格納するためのリスト
properties_list = []

# 空のデータフレームを作成
df = pd.DataFrame()

# 物件情報が含まれる要素をすべて取得
properties = soup.find_all('div', class_='property')

In [29]:
# 物件情報の抽出
for prop in properties:
    # 物件名が記載されているclassを選択
    title = prop.find('h2', class_='property_inner-title').text.strip()
    property_link = prop.find('a', class_='js-cassetLinkHref')['href']

    # 取り扱い店舗が記載されているclassを選択
    property_stores = prop.find('a', class_='js-noCassetteLink').text.strip()
    property_stores_link = prop.find('a', class_='js-noCassetteLink')['href']

    # 住所・アクセス経路・賃料・管理費・間取り・専有面積・向き・築年数・マンションかアパート・敷金・礼金が記載されているtdタグを選択
    detailbox = prop.find('div', class_='detailbox')
    if detailbox:

        # アクセス経路が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        access_element = prop.find('div', class_='detailnote-box').text.strip()

        access_elements = access_element.split('\n')
        access_1 = access_elements[0].strip() if len(access_elements) > 0 else None
        access_2 = access_elements[1].strip() if len(access_elements) > 1 else None
        access_3 = access_elements[2].strip() if len(access_elements) > 2 else None

        # 賃料・管理費が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        fee_element = detailbox.find_all('td', class_='detailbox-property-col')[0].text.strip()
        rent_price = fee_element.split('\n')[0].strip()
        management_fee =  fee_element.split('\n')[1].strip()

        # 敷金・礼金が記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        security_deposit_element = detailbox.find_all('td', class_='detailbox-property-col')[1].text.strip()
        security_deposit = security_deposit_element.split('\n')[0].strip()
        key_money = security_deposit_element.split('\n')[1].strip()

        # 間取り・専有面積・向きが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        house_layout_element = detailbox.find_all('td', class_='detailbox-property-col')[2].text.strip()
        house_layout =  house_layout_element.split('\n')[0].strip()
        exclusive_area =  house_layout_element.split('\n')[2].strip()
        direction =  house_layout_element.split('\n')[4].strip()

        # 築年数とマンションかアパートが記載されているtdタグを選択し、改行文字で分割して必要な部分を取得
        building_type_element = detailbox.find_all('td', class_='detailbox-property-col')[3].text.strip()
        building_type =  building_type_element.split('\n')[0].strip()
        building_age = building_type_element.split('\n')[2].strip()

        # 住所が記載されているtdタグを正確に選択
        address = detailbox.find_all('td', class_='detailbox-property-col')[4].text.strip()

    # データリストに情報を追加
    properties_list.append({
        '物件名': title,
        'URL': "https://suumo.jp"+property_link,
        '住所': address,
        'アクセス1': access_1,
        'アクセス2': access_2,
        'アクセス3': access_3,
        '築年数': building_age,
        '家賃': rent_price,
        '管理費': management_fee,
        '間取り': house_layout,
        '専有面積': exclusive_area,        
        '向き': direction,
        'タイプ': building_type,
        '敷金': security_deposit,
        '礼金': key_money,
        '取り扱い店舗': property_stores,
        '取り扱い店舗URL': "https://suumo.jp"+property_stores_link
    })

In [30]:
# データフレームの作成
df = pd.DataFrame(properties_list)

In [31]:
# 区のカラムを作成
df['区'] = df["住所"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [34]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

df['家賃'] = df['家賃'].apply(change_fee)
#df['敷金'] = df['敷金'].apply(change_fee)
#df['礼金'] = df['礼金'].apply(change_fee)

TypeError: argument of type 'float' is not iterable

In [35]:
# データフレームの表示（省略せずに全て表示する設定も可能です）
df.head()

,物件名,URL,住所,アクセス1,アクセス2,アクセス3,築年数,家賃,管理費,間取り,専有面積,向き,タイプ,敷金,礼金,取り扱い店舗,取り扱い店舗URL,区
0,Ｍｏｎｋｓｔｏｗｎ夏目坂 104号室,https://suumo.jp/chintai/bc_100375809678/,東京都新宿区戸山１,東京メトロ東西線/早稲田駅 歩4分,都営大江戸線/若松河田駅 歩10分,都営大江戸線/牛込柳町駅 歩12分,新築,28.5,管理費 10000円,12LDK,75.09m2,-,マンション,敷28.5万円,礼28.5万円,(株)Street2,https://suumo.jp/chintai/kaisha/kc_030_170304001/,新宿区
1,レジディア市谷砂土原 Ｓ,https://suumo.jp/chintai/bc_100382140932/,東京都新宿区市谷砂土原町３,都営大江戸線/牛込神楽坂駅 歩6分,東京メトロ東西線/神楽坂駅 歩13分,ＪＲ中央線/市ケ谷駅 歩14分,築17年,115.0,管理費 -,4LDK,180.44m2,南,マンション,敷345万円,礼-,いい部屋ネット大東建託リーシング(株)渋谷店,https://suumo.jp/chintai/kaisha/kc_030_159885726/,新宿区
2,クレストレジデンス東中野,https://suumo.jp/chintai/bc_100382103064/,東京都新宿区北新宿３,ＪＲ中央線/東中野駅 歩4分,ＪＲ山手線/新大久保駅 歩18分,ＪＲ中央線/大久保駅 歩14分,築16年,28.0,管理費 20000円,3LDK,70.19m2,西,マンション,敷28万円,礼28万円,アエラス目白店 (株)アエラス.ER,https://suumo.jp/chintai/kaisha/kc_030_149891011/,新宿区
3,メゾンカルム西新宿,https://suumo.jp/chintai/bc_100382193503/,東京都新宿区西新宿４,都営大江戸線/西新宿五丁目駅 歩5分,京王新線/初台駅 歩11分,東京メトロ丸ノ内線/西新宿駅 歩16分,築15年,20.1,管理費 9000円,2LDK,50.07m2,南東,マンション,敷20.1万円,礼20.1万円,いい部屋ネット大東建託リーシング(株)池袋東口店,https://suumo.jp/chintai/kaisha/kc_030_159885741/,新宿区
4,アバンテ西落合 D号室,https://suumo.jp/chintai/bc_100382249701/,東京都新宿区西落合２,都営大江戸線/落合南長崎駅 歩7分,西武池袋線/東長崎駅 歩12分,西武新宿線/新井薬師前駅 歩14分,築15年,25.0,管理費 -,3LDK,81.96m2,西,一戸建て,敷-,礼25万円,(有)リバーハウジング,https://suumo.jp/chintai/kaisha/kc_030_153847003/,新宿区


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   物件名        30 non-null     object 
 1   URL        30 non-null     object 
 2   住所         30 non-null     object 
 3   アクセス1      30 non-null     object 
 4   アクセス2      30 non-null     object 
 5   アクセス3      30 non-null     object 
 6   築年数        30 non-null     object 
 7   家賃         30 non-null     float64
 8   管理費        30 non-null     object 
 9   間取り        30 non-null     object 
 10  専有面積       30 non-null     object 
 11  向き         30 non-null     object 
 12  タイプ        30 non-null     object 
 13  敷金         30 non-null     object 
 14  礼金         30 non-null     object 
 15  取り扱い店舗     30 non-null     object 
 16  取り扱い店舗URL  30 non-null     object 
 17  区          30 non-null     object 
dtypes: float64(1), object(17)
memory usage: 4.3+ KB


In [37]:
def save_to_database(quotes, db_name=None):
    if db_name is None:
        date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        db_name = f'Property_data_{date_str}.db'
    try:
        df = pd.DataFrame(quotes)
        conn = sqlite3.connect(db_name)
        
        # テーブルが存在しない場合は作成する
        conn.execute('''CREATE TABLE IF NOT EXISTS Property_data
                        ({})'''.format(', '.join([f"{col} TEXT" for col in df.columns])))
        
        df.to_sql('Property_data', conn, if_exists='append', index=False)
        conn.close()
        logging.info(f"Data successfully saved to {db_name}")
    except Exception as e:
        logging.error(f"Error saving data to database: {e}")

In [38]:
#DBファイルとして保存する関数
def create_and_save_to_db(quotes, columns, db_name=None):
    if db_name is None:
        date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        db_name = f'Property_data_{date_str}.db'

    try:
        df = pd.DataFrame(quotes)
        conn = sqlite3.connect(db_name)

        # テーブルが存在しない場合は作成する
        conn.execute('''CREATE TABLE IF NOT EXISTS Property_data
                        ({})'''.format(', '.join([f"{col} TEXT" for col in columns])))

        df.to_sql('Property_data', conn, if_exists='append', index=False)
        conn.close()
        logging.info(f"Data successfully saved to {db_name}")

    except Exception as e:
        logging.error(f"Error saving data to database: {e}")

In [39]:
# データベース名を指定
db_name = 'estate_list.db'  # ここに実際のデータベースファイル名を入力してください

save_to_database(df, db_name)

2024-05-21 00:30:38,886 - INFO - Data successfully saved to estate_list.db


In [13]:
# データベース名を指定
db_name = 'estate_list.db'  # ここに実際のデータベースファイル名を入力してください

#create_and_save_to_db(df, db_name)

In [14]:
# 作成したDBファイルを読み込む
conn = sqlite3.connect(db_name)
df2 = pd.read_sql('SELECT * FROM Property_data', conn)
conn.close()

In [15]:
df2.head()

,物件名,URL,住所,アクセス1,アクセス2,アクセス3,築年数,賃貸料,管理費,間取り,専有面積,向き,タイプ,敷金,礼金,取り扱い店舗,取り扱い店舗URL,区
0,Ｍｏｎｋｓｔｏｗｎ夏目坂 104号室,https://suumo.jp/chintai/bc_100375809678/,東京都新宿区戸山１,東京メトロ東西線/早稲田駅 歩4分,都営大江戸線/若松河田駅 歩10分,都営大江戸線/牛込柳町駅 歩12分,新築,28.5万円,管理費 10000円,12LDK,75.09m2,-,マンション,敷28.5万円,礼28.5万円,(株)Street2,https://suumo.jp/chintai/kaisha/kc_030_170304001/,新宿区
1,レジディア市谷砂土原 Ｓ,https://suumo.jp/chintai/bc_100382140932/,東京都新宿区市谷砂土原町３,都営大江戸線/牛込神楽坂駅 歩6分,東京メトロ東西線/神楽坂駅 歩13分,ＪＲ中央線/市ケ谷駅 歩14分,築17年,115万円,管理費 -,4LDK,180.44m2,南,マンション,敷345万円,礼-,いい部屋ネット大東建託リーシング(株)渋谷店,https://suumo.jp/chintai/kaisha/kc_030_159885726/,新宿区
2,クレストレジデンス東中野,https://suumo.jp/chintai/bc_100382103064/,東京都新宿区北新宿３,ＪＲ中央線/東中野駅 歩4分,ＪＲ山手線/新大久保駅 歩18分,ＪＲ中央線/大久保駅 歩14分,築16年,28万円,管理費 20000円,3LDK,70.19m2,西,マンション,敷28万円,礼28万円,アエラス目白店 (株)アエラス.ER,https://suumo.jp/chintai/kaisha/kc_030_149891011/,新宿区
3,メゾンカルム西新宿,https://suumo.jp/chintai/bc_100382193503/,東京都新宿区西新宿４,都営大江戸線/西新宿五丁目駅 歩5分,京王新線/初台駅 歩11分,東京メトロ丸ノ内線/西新宿駅 歩16分,築15年,20.1万円,管理費 9000円,2LDK,50.07m2,南東,マンション,敷20.1万円,礼20.1万円,いい部屋ネット大東建託リーシング(株)池袋東口店,https://suumo.jp/chintai/kaisha/kc_030_159885741/,新宿区
4,アバンテ西落合 D号室,https://suumo.jp/chintai/bc_100382249701/,東京都新宿区西落合２,都営大江戸線/落合南長崎駅 歩7分,西武池袋線/東長崎駅 歩12分,西武新宿線/新井薬師前駅 歩14分,築15年,25万円,管理費 -,3LDK,81.96m2,西,一戸建て,敷-,礼25万円,(有)リバーハウジング,https://suumo.jp/chintai/kaisha/kc_030_153847003/,新宿区
